In [1]:
%matplotlib notebook

import numpy as np
import astropy.units as u
from astropy import time
from astropy.time import Time
from astropy.time import TimeDelta
from astropy.coordinates import solar_system_ephemeris
from poliastro import iod
from poliastro.bodies import Sun
from poliastro.twobody import Orbit
from poliastro.ephem import get_body_ephem
from poliastro.constants import GM_mars, GM_earth, R_mars, R_earth
from timeit import default_timer as timer


solar_system_ephemeris.set("jpl")
h = 390  
 
date_launch_1 = time.Time('2018-01-03 12:00', scale='utc')

def velocity(number):
    start = timer()
    DeltaDL = 0
    y = []
    z = []
    q = 0
    p = 0
    s = 0
    f = 0
    b = 0
    flight_1 = []
    flight_2 = []
    dvbudget = (number/1000) * u.km / u.s # determines delta V in km/s
    
    
    # to Mars
    
    for DeltaDL in range (0, 200, 2):  # h gives the range of loop
        
        DeltaL = TimeDelta(DeltaDL, format = 'jd') # l gives the change in arrival date
        date_launch = date_launch_1 + DeltaL # sets time of arrival
        
        
        
        for tof in range(70, 200, 2): # range of tof loop
        
            TOF = TimeDelta(tof, format='jd')
            date_arrival = date_launch + TOF # sets time of launch
            timeMiss = TimeDelta(240, format='sec') # corresponds to miss distance
        
            r0, vv_e = get_body_ephem("earth", date_launch) 
            vv_e  = vv_e.to(u.km / u.second)
            rf, vv_m = get_body_ephem("mars", (date_arrival + timeMiss)) 
            vv_m  = vv_m.to(u.km / u.second)
            #planet velocity and position vecors
            
            (va, vb), = iod.lambert(Sun.k, r0, rf, TOF) # solving LAMBERT'S PROBLEM
            ss0_trans = Orbit.from_vectors(Sun, r0, va, date_launch) # creating orbit         
            incl = ss0_trans.inc # inclination
            e = ss0_trans.ecc    # eccentricity
            a = ss0_trans.a      # semimajor axis
            
            # accounting for Earth Orbit
            v_1 = va - vv_e
            dv1 = np.sqrt(v_1.dot(v_1)) # magnitude                    
            radius_E_i = (R_earth.value + 1000*1000)
            v_parking_E  = np.sqrt(GM_earth.value/radius_E_i)*(u.m/u.s) #velocity in parking orbit
            v_hyperbolic_E = np.sqrt((dv1.value*1000)**2+(2*GM_earth.value)/(radius_E_i))*(u.m/u.s) 
            dv_1 = v_hyperbolic_E - v_parking_E             
            E_Hyperbolic_incl = np.sin(incl/2)**2 
            Real_Delta_v_E =  np.sqrt(dv_1**2+(4*v_parking_E*v_hyperbolic_E*E_Hyperbolic_incl))
            # = delta v accounting for change in incl
            
            
            # accounting for Mars Orbit
            v_2 = vv_m - vb
            dv2 = np.sqrt(v_2.dot(v_2)) #magnitudes        
            Radius_Parking_M = (R_mars.value + 250*1000)
            v_parking_M  = np.sqrt(GM_mars.value/ Radius_Parking_M )*u.m/u.s
            v_hyperbolic_M = np.sqrt((dv2.value*1000)**2+(2*GM_mars.value)/( Radius_Parking_M ))*u.m/u.s
            dv_2 = v_hyperbolic_M - v_parking_M #delta v to get in circular orbit
            M_Hyperbolic_incl = np.sin(incl/2)**2
            Real_Delta_v_M =  np.sqrt(dv_2**2+(4*v_parking_M*v_hyperbolic_M*M_Hyperbolic_incl))
                                      
            tdv = (  Real_Delta_v_E + Real_Delta_v_M )
            q += 1
            print(q,end=' \r')
            
            if (dvbudget - (5 *  u.km / u.s) )  >= tdv:
                y = np.array([date_arrival, tof, tdv])
                flight_1 = np.append(flight_1,[y[:]])
           
        
        
        

            
     
    end = timer()
    print(end - start)
    # to Earth  
    for WaitTime in range(0, 200, 2):
        WT = TimeDelta(WaitTime, format = 'jd')
        date_launch_2 = time.Time('2018-01-03 12:00', scale='utc') + TimeDelta(90, format = 'jd')

        for tof2 in range(70, 200, 2):

            TOF_2 = TimeDelta(tof2, format='jd')
            date_arrival_2 = date_launch_2 + TOF_2
            timeMiss_2 = TimeDelta(240, format='sec')                        


            r0_2, vv_m_2 = get_body_ephem("mars", date_launch_2) 
            vv_m_2  = vv_m_2.to(u.km / u.second)
            rf_2, vv_e_2 = get_body_ephem("earth", (date_arrival_2 + timeMiss_2)) 
            vv_e_2  = vv_e_2.to(u.km / u.second)
            #planet velocity and position vecors

            (va_2, vb_2), = iod.lambert(Sun.k, r0_2, rf_2, TOF_2) # solving LAMBERT'S PROBLEM
            ss0_trans_2 = Orbit.from_vectors(Sun, r0_2, va_2, date_launch_2) # creating orbit         
            incl_2 = ss0_trans_2.inc # inclination
            e_2 = ss0_trans_2.ecc    # eccentricity
            a_2 = ss0_trans_2.a      # semimajor axis



            # accounting for Mars Orbit
            v_3 = vv_m_2 - va_2
            dv3 = np.sqrt(v_3.dot(v_3)) # magnitude                    
            radius_M_i = (R_mars.value + 250*1000)
            v_parking_M_2  = np.sqrt(GM_mars.value/radius_M_i)*(u.m/u.s) #velocity in parking orbit
            v_hyperbolic_M_2 = np.sqrt((dv3.value*1000)**2+(2*GM_mars.value)/(radius_M_i))*(u.m/u.s) 
            dv_3 = v_hyperbolic_M_2 - v_parking_M_2            
            M_Hyperbolic_incl_2 = np.sin(incl_2/2)**2 
            Real_Delta_v_M_2 =  np.sqrt(dv_3**2+(4*v_parking_M_2*v_hyperbolic_M_2*M_Hyperbolic_incl_2))
            # = delta v accounting for change in incl


            # accounting for Mars Orbit
            v_4 = vb_2 - vv_e_2 
            dv4 = np.sqrt(v_4.dot(v_4)) #magnitudes        
            Radius_Parking_E = (R_earth.value + 1000*1000)
            v_parking_E_2  = np.sqrt(GM_earth.value/ Radius_Parking_E )*u.m/u.s
            v_hyperbolic_E_2 = np.sqrt((dv4.value*1000)**2+(2*GM_earth.value)/( Radius_Parking_E ))*u.m/u.s
            dv_4 = v_hyperbolic_E_2 - v_parking_E_2 #delta v to get in circular orbit
            E_Hyperbolic_incl_2 = np.sin(incl_2/2)**2
            Real_Delta_v_E_2 =  np.sqrt(dv_4**2+(4*v_parking_E_2*v_hyperbolic_E_2*E_Hyperbolic_incl_2))

            tdv_2 = Real_Delta_v_E_2 + Real_Delta_v_M_2
            f += 1
            print(f,end=' \r')
            if (dvbudget - (5 *  u.km / u.s) )  >= tdv_2:
                z = np.array([date_arrival_2, tof2, tdv_2])
                flight_2 = np.append(flight_2,[z[:]])
                
    end = timer()
    print(end - start)
    
    velocity.flight_1 = flight_1
    velocity.flight_2 = flight_2

In [2]:
velocity(20500)

210.69529080076995                                                                                                                                                                                                                                              
417.51496676946107                                                                                                                                                                                                                                                                                          


In [1]:
def linkvectors(vect_1, vect_2):
    vect_1 = vect_1.reshape( int((vect_1.size)/3) , 3)
    vect_1 =  vect_1[np.argsort(vect_1[:,0])]
    
    vect_2 = vect_2.reshape( int((vect_2.size)/3) , 3)
    vect_2 =  vect_2[np.argsort(vect_2[:,0])]
    
    v1 = []
    v2 = []
    q = 0
    
    print((vect_2.size*vect_1.size)/9)
    for x in range (0, int((vect_2.size)/3), 1):
        
        if ( vect_2[x,0] - TimeDelta(20, format = 'jd') ) >= vect_1[0,0]:
            if ( vect_2[x,0] - TimeDelta(100, format = 'jd') ) <= vect_1[0,0]:                
                v2 = np.append(v2,[vect_2[x,:]])
             
    print((v2.size*vect_1.size)/9)
    v2 = v2.reshape( int((v2.size)/3) , 3)
    
    for y in range(0, int((v2.size)/3), 1):
        for z in range(0, int((vect_1.size)/3), 1):
            q += 1
            print(q,end=' \r')
            if ( v2[y,0] - TimeDelta(20, format = 'jd') ) >= vect_1[z,0]:
                if ( v2[y,0] - TimeDelta(100, format = 'jd') ) <= vect_1[z,0]: 
                    
                    if ( v2[y,2] + vect_1[z,2]) <= 20.5 * u.km / u.s:
                        c = v2[y,2] + vect_1[z,2]
                        v1 = np.append(v1,[vect_2[y,:],vect_1[z,:]],c)
    
    v1 =  v1[np.argsort(v1[:,5])]
    
    linkvectors.v1 = v1
        
        
                
            
    
    
    

In [ ]:
linkvectors(velocity.flight_1, velocity.flight_2)